# Challenge Data Science

Desafio [Alura](https://www.alura.com.br/) -  Um projeto de Data Science com PySpark


##  Semana 02

### Imports e carregamento dos Dados

In [1]:
from pyspark.sql import(
    SparkSession, 
    functions as F,
    DataFrame
) 
import utils

In [2]:
spark = (
    SparkSession
    .builder
    .appName('InsightPlaces-Semana_02')
    .getOrCreate()
)

spark

22/12/12 20:55:21 WARN Utils: Your hostname, lenovo-pessoal resolves to a loopback address: 127.0.1.1; using 192.168.0.233 instead (on interface wlp0s20f3)
22/12/12 20:55:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 20:55:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/12 20:55:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data_imoveis = (
    spark.read.format('parquet')
    .load('data/semana_02/dataset_tratado/')
)


### Exploração dos Dados

In [4]:
data_imoveis.show(5)

+-----+----------+---------+---------+--------------------+---------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|  valor|
+-----+----------+---------+---------+--------------------+---------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-------+
|    3|      43.0|     43.0|      1.0|Academia - Churra...|Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|     245.0|null|Venda|15000.0|
|    2|      42.0|     42.0|      1.0|Churrasqueira - P...|Paciência|Zona Oeste|085bab2c-87ad-452...|    2.0|  null|       Usado| Apartamento|Residencial| 1.0|       0.0| 0.0|Venda|15000.0|
|    1|      41.0|     41.0|      1.0|Portaria 24h

In [5]:
data_imoveis.groupBy('zona').count().show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
|            |   11|
+------------+-----+



In [6]:
''' Deletando colunas tipo_anuncio, tipo_unidade e tipo_uso, pois cada uma
delas possuem valores únicos.

As Colunas area_total e area_util praticamente são iguais e por isso apenas
uma delas será mantida no dataFrame.

'''

data_imoveis = (
    data_imoveis
    .drop('tipo_anuncio')
    .drop('tipo_unidade')
    .drop('tipo_uso')
    .drop('area_total')
)


data_imoveis.printSchema()
data_imoveis.show(5)

root
 |-- andar: long (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: double (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- id_anuncio: string (nullable = true)
 |-- quartos: double (nullable = true)
 |-- suites: double (nullable = true)
 |-- vaga: double (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)

+-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+
|andar|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|vaga|condominio|iptu| tipo|  valor|
+-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+
|    3|     43.0|      1.0|

In [7]:
data_imoveis.select('caracteristicas').distinct().show(10, truncate=False)

# 6975

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|caracteristicas                                                                                                                                          |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|Churrasqueira - Condomínio fechado - Portão eletrônico                                                                                                   |
|Playground - Condomínio fechado - Portaria 24h - Animais permitidos                                                                                      |
|Piscina - Academia - Churrasqueira - Playground - Elevador - Condomínio fechado - Portão eletrônico - Portaria 24h - Animais permitidos                  |
|Elevador - Playground - Piscina - Salão de festas - Condomínio 

#### Tratamento da coluna caracteristicas

A colluna caracteristicas tem é uma string com itens separados por ' - ', desta forma será montado um dataframe apenas com está coluna para em seguida agrupar e exibir cada palavra unica

In [8]:
df_caracteristicas = data_imoveis.select('caracteristicas').distinct()

In [9]:
words = df_caracteristicas.select(F.explode( F.split(df_caracteristicas.caracteristicas, ' - ') ).alias('word') )

# 48149

In [10]:
# Palavras unicas da coluna característica

words.groupBy('word').count().show()

+------------------+-----+
|              word|count|
+------------------+-----+
|Condomínio fechado| 5557|
|        Playground| 4839|
| Portão eletrônico| 3859|
|           Piscina| 5016|
|Animais permitidos| 4555|
|      Portaria 24h| 4278|
|          Elevador| 5710|
|          Academia| 4328|
|   Salão de festas| 5407|
|                  |    1|
|     Churrasqueira| 4599|
+------------------+-----+



#### Adicionando features

In [11]:
data_imoveis = (
    data_imoveis
    .withColumn('condominio_fechado', F.when(data_imoveis.caracteristicas.like('%Condomínio fechado%'), 1 ).otherwise(0) )
    .withColumn('playground', F.when(data_imoveis.caracteristicas.like('%Playground%'), 1 ).otherwise(0) )
    .withColumn('portao_eletronico', F.when(data_imoveis.caracteristicas.like('%Portão eletrônico%'), 1 ).otherwise(0) )
    .withColumn('piscina', F.when(data_imoveis.caracteristicas.like('%Piscina%'), 1 ).otherwise(0) )
    .withColumn('permite_animais', F.when(data_imoveis.caracteristicas.like('%Animais permitidos%'), 1 ).otherwise(0) )
    .withColumn('portaria_24h', F.when(data_imoveis.caracteristicas.like('%Portaria 24h%'), 1 ).otherwise(0) )
    .withColumn('elevador', F.when(data_imoveis.caracteristicas.like('%Elevador%'), 1 ).otherwise(0) )
    .withColumn('academia', F.when(data_imoveis.caracteristicas.like('%Academia%'), 1 ).otherwise(0) )
    .withColumn('salao_de_festa', F.when(data_imoveis.caracteristicas.like('%Salão de festas%'), 1 ).otherwise(0) )
    .withColumn('churrasqueira', F.when(data_imoveis.caracteristicas.like('%Churrasqueira%'), 1 ).otherwise(0) )
)

data_imoveis.show(5)
# +-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+
# |andar|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|vaga|condominio|iptu| tipo|  valor|condominio_fechado|playground|portao_eletronico|piscina|permite_animais|portaria_24h|elevador|academia|salao_de_festa|churrasqueira|
# +-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+
# |    3|     43.0|      1.0|Academia - Churra...|Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|    2.0|  null| 1.0|     245.0|null|Venda|15000.0|                 1|         1|                1|      0|              1|           1|       0|       1|             1|            1|
# |    2|     42.0|      1.0|Churrasqueira - P...|Paciência|Zona Oeste|085bab2c-87ad-452...|    2.0|  null| 1.0|       0.0| 0.0|Venda|15000.0|                 1|         1|                1|      0|              1|           1|       0|       0|             1|            1|
# |    1|     41.0|      1.0|Portaria 24h - Co...|Guaratiba|Zona Oeste|18d22cbe-1b86-476...|    2.0|  null| 1.0|       0.0| 0.0|Venda|20000.0|                 1|         1|                1|      0|              0|           1|       0|       0|             1|            1|
# |    3|     43.0|      1.0|Churrasqueira - P...|   Cosmos|Zona Oeste|bed8a354-9317-442...|    2.0|  null| 0.0|     285.0|null|Venda|20000.0|                 1|         1|                1|      1|              1|           1|       0|       0|             1|            1|
# |    2|     43.0|      1.0|Academia - Churra...|Guaratiba|Zona Oeste|12a13315-d67f-48f...|    2.0|  null| 1.0|     245.0|null|Venda|15000.0|                 1|         1|                1|      0|              1|           1|       0|       1|             1|            1|
# +-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+

+-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+
|andar|area_util|banheiros|     caracteristicas|   bairro|      zona|          id_anuncio|quartos|suites|vaga|condominio|iptu| tipo|  valor|condominio_fechado|playground|portao_eletronico|piscina|permite_animais|portaria_24h|elevador|academia|salao_de_festa|churrasqueira|
+-----+---------+---------+--------------------+---------+----------+--------------------+-------+------+----+----------+----+-----+-------+------------------+----------+-----------------+-------+---------------+------------+--------+--------+--------------+-------------+
|    3|     43.0|      1.0|Academia - Churra...|Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|    2.0|  null| 1.0|     245.0|null|Venda|15000.0|                 1|         1|           

In [12]:
data_imoveis.select('playground').groupBy('playground').count().show()

+----------+-----+
|playground|count|
+----------+-----+
|         1|31856|
|         0|34706|
+----------+-----+



#### Tratando dados nulos e NaN's